In [1]:
import time
from parsel import Selector
from selenium import webdriver
# from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
# from selenium.common.exceptions import NoSuchElementException
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service as ChromeService

In [2]:
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [3]:
data_dir = '../data/'

In [4]:
# configure webdriver
options = Options()
options.h(True)
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")

# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    # this will disable image loading
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

<ipython-input-4-aa7710fbb3d2>:3: DeprecationWarning: use setter for headless property instead of set_headless
  options.set_headless(True)


In [10]:
#Initialize data structures
game_list = {'id':[], 'name':[], 'developer':[], 'release_date': [], 'metascore':[], 'review_platform': [], 'other_platforms': [], 'genres': [], 'summary': [],
            'online_player_num':[], 'rating': []}

critic_reviews = {}

#Initialize chrome webdriver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options, chrome_options=chrome_options)

driver.get("https://www.metacritic.com/browse/games/score/metascore/year/pc/filtered?year_selected=2021&view=detailed&sort=desc&page=0")

game_id = 0
to = 7

while True:
    
    # Games listed on page
    home = driver.current_url
    page_games = driver.find_elements_by_class_name('clamp-summary-wrap')
    game_num = 0
    
    while game_num < len(page_games):
        page_games = driver.find_elements_by_class_name('clamp-summary-wrap')
        game_link = page_games[game_num].find_element_by_tag_name('h3')
        game_link.click()
        
        # Game Details
        det = WebDriverWait(driver, timeout=to).until(lambda d: d.find_element_by_link_text('Details & Credits'))
        det.click()
        
        # Fill in game attributes
        game_list['id'].append(game_id)

        game_list['name'].append(
            WebDriverWait(driver, timeout=to).until(lambda d: d.find_element_by_tag_name('h1').text)
        )
        game_list['release_date'].append(
            WebDriverWait(driver, timeout=to).until(lambda d: d.find_element_by_xpath('//*[@id="main"]/div[1]/div[3]/ul/li[2]/span[2]').text)
        )
        game_list['metascore'].append(
            driver.find_element_by_xpath('//*[@id="main"]/div[3]/div/div[2]/div[1]/div[1]/div/div/a/div/span').text
        )
        game_list['review_platform'].append(
            driver.find_element_by_xpath('//*[@id="main"]/div[1]/div[2]/span/a').text
        )
        
        try:
            game_list['other_platforms'].append(
                driver.find_element_by_xpath('//*[@id="main"]/div[1]/div[3]/ul/li[3]/span[2]').text
            )
        except:
            game_list['other_platforms'].append("")
        
        game_list['summary'].append(
            driver.find_element_by_xpath('//*[@id="main"]/div[3]/div/div[2]/div[2]/div/span[2]').text
        )
        
        table = driver.find_elements_by_class_name('product_details')
        rows = table[1].find_elements_by_tag_name('tr')
        
        table_dict = {}
        for row in rows:
            key = row.find_element_by_tag_name('th').text
            detail = row.find_element_by_tag_name('td').text
            table_dict[key] = detail
        
        try:
            game_list['rating'].append(table_dict['Rating:'])
        except:
            game_list['rating'].append("")
        
        try:
            game_list['developer'].append(table_dict['Developer:'])
        except:
            game_list['developer'].append("")
            
        try:
            game_list['genres'].append(table_dict['Genre(s):'])
        except:
            game_list['genres'].append("")

        try:
            game_list['online_player_num'].append(table_dict['Number of Online Players:'])
        except:
            game_list['online_player_num'].append("")
        
        game_num+=1
        game_id+=1
        driver.get(home)
        
    # Go to next page
    try:
        next_button = driver.find_element_by_link_text('next')
        next_button.click()
    except:
        break



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\siraj\.wdm\drivers\chromedriver\win32\98.0.4758.80\chromedriver.exe] found in cache
<ipython-input-10-5b53eee6a804>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options, chrome_options=chrome_options)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=98.0.4758.82)


In [ ]:
next_button = driver.find_element_by_link_text('next')
next_button.click()

In [8]:
import pandas as pd

pd.DataFrame(game_list)

ValueError: arrays must all be same length

In [ ]:
print(table_dict)